# Fake News Generation

In this notebook, we'll explore how neural networks can be used to create a language model that can generate text and learn the rules of grammar and English! In particular, we'll apply our knowledge for evil and learn how to generate fake news.

In this notebook we'll be:
1.   Exploring and Implementing Language Models



In [1]:
#@title Run this cell to import libraries and download the data! If there is a prompt, just enter "A"
import os
import random
import string
import sys
from ipywidgets import interact

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense

import gdown
import warnings
warnings.filterwarnings('ignore')
# gdown.download("https://drive.google.com/uc?id=11WClewW80aEj8RrdmS9qkchwQsOkJlHy", 'fake.txt', True)
# gdown.download("https://drive.google.com/uc?id=1UuANHblVzkclCC2v9J0V7uxX0Y0Fjfkx", 'pre_train.zip', True)
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Fake%20News%20Detection/fake.txt'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Fake%20News%20Detection/pre_train.zip'
! unzip -oq pre_train.zip

--2023-08-09 20:59:54--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Fake%20News%20Detection/fake.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 142.251.16.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300000 (293K) [text/plain]
Saving to: ‘fake.txt’

fake.txt            100%[===================>] 292.97K  --.-KB/s    in 0.003s  

2023-08-09 20:59:54 (95.7 MB/s) - ‘fake.txt’ saved [300000/300000]

--2023-08-09 20:59:55--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Fake%20News%20Detection/pre_train.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 142.251.16.128, ...
Connecting to storage.googleapis.com (storage.go

In [2]:
#@title Run this cell to load some helper functions
def load_data():
    with open("fake.txt", "r") as f:
        return f.read()

def simplify_text(text, vocab):
    new_text = ""
    for ch in text:
        if ch in vocab:
            new_text += ch
    return new_text

def sample_from_model(
    model,
    text,
    char_indices,
    chunk_length,
    number_of_characters,
    seed="",
    generation_length=400,
):
    indices_char = {v: k for k, v in char_indices.items()}
    for diversity in [0.2, 0.5, 0.7]:
        print("----- diversity:", diversity)
        generated = ""
        if not seed:
            text = text.lower()
            start_index = random.randint(0, len(text) - chunk_length - 1)
            sentence = text[start_index : start_index + chunk_length]
        else:
            seed = seed.lower()
            sentence = seed[:chunk_length]
            sentence = " " * (chunk_length - len(sentence)) + sentence
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for _ in range(generation_length):
            x_pred = np.zeros((1, chunk_length, number_of_characters))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print("\n")


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64") + 1e-8
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


class SampleAtEpoch(tf.keras.callbacks.Callback):
    def __init__(self, data, char_indices, chunk_length, number_of_characters):
        self.data = data
        self.char_indices = char_indices
        self.chunk_length = chunk_length
        self.number_of_characters = number_of_characters
        super().__init__()

    def on_epoch_begin(self, epoch, logs=None):
        sample_from_model(
            self.model,
            self.data,
            self.char_indices,
            self.chunk_length,
            self.number_of_characters,
            generation_length=200,
        )


def predict_str(model, text, char2indices, top=10):
    if text == '':
      print("waiting...")
      return
    text = text.lower()
    assert len(text) < CHUNK_LENGTH
    oh = np.array([one_hot_sentence(text, char2indices)])
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      pred = model.predict(oh).flatten()
    sort_indices = np.argsort(pred)[::-1][:top]
    plt.bar(range(top), pred[sort_indices], tick_label=np.array(list(VOCAB))[sort_indices])
    plt.title(f"Predicted probabilities of the character following '{text}'")
    plt.show()

## Language models

A language model tries to learn how language works. Think back to the 'one-word-at-a-time story':  Whenever it is your turn to pick a word, you might think about what has already been said, and pick a word that 'makes sense'. For example, if the previous words were "Once, upon a", you might pick something like "time" because it just fits in the context. Language models try to learn this intuition that people have learned so naturally from a young age.

Our language model today will look at the previous words in a sequence and use that compute the probabilities of what the next word will be. Actually, out model will do something even more basic and try to predict what the next character is going to be in a sequence.

The next cell defines some constants that we'll be using in our language model

*   `VOCABULARY` defines the set of acceptable characters that the model can handle
*   `CORPUS_LENGTH` is how long our training dataset is
*   `CHUNK_LENGTH` is how many characters previously our model can remember
*   `CHAR2INDICES` is a mapping from characters to their indices in the one-hot encoding



In [3]:
STEP = 3
LEARNING_RATE = 0.0005
CORPUS_LENGTH = 200000
CHUNK_LENGTH = 40
VOCAB = string.ascii_lowercase + string.punctuation + string.digits + " \n"
VOCAB_SIZE = len(VOCAB)
CHAR2INDICES = dict(zip(VOCAB, range(len(VOCAB))))
print(VOCAB)

abcdefghijklmnopqrstuvwxyz!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~0123456789 



Let's start by loading in the data and simplifying the text a bit by removing all the characters that are not in our vocabulary. Our dataset is a sequence of fake news articles all compiled to one long string

In [4]:
data = load_data()
data = data[:CORPUS_LENGTH]
data = simplify_text(data, CHAR2INDICES)
print(f"Type of the data is: {type(data)}\n")
print(f"Length of the data is: {len(data)}\n")
print(f"The first couple of sentence of the data are:\n")
print(data[:500])

Type of the data is: <class 'str'>

Length of the data is: 200000

The first couple of sentence of the data are:

print they should pay all the back all the money plus interest. the entire family and everyone who came in with them need to be deported asap. why did it take two years to bust them? 
here we go again another group stealing from the government and taxpayers! a group of somalis stole over four million in government benefits over just 10 months! 
weve reported on numerous cases like this one where the muslim refugees/immigrants commit fraud by scamming our systemits way out of control! more relate


## Encoding words

We are happy to read words like above, but like we mentioned in lecture, computers prefer numbers. So we'll have to do some processing to our data. Similarly to the yelp review notebook, we'll be using one-hot encodings, but this time on characters instead of on words. Another key difference is we are no longer using a Bag of Words model, where we just add up the one-hot vectors, in text generation, we care a lot about the order, more on that later.



### Exercise 1a
<b>Task:</b> Complete the implementation of the `one_hot` function, which creates a one-hot vector for a single character.

<b>Inputs:</b>
* `char`: A single character
* `char_indices`: Stores the mapping between characters and indices.

<b>Output:</b>
* `vec`: A one-hot vector for `char`.

Remember that a one-hot vector is a list with zeros everywhere, except a 1 in the index for that character.

In [7]:
def one_hot(char, char_indices):
    num_chars = len(char_indices)
    vec = np.zeros(num_chars) # Use numpy to create a vector of all 0s
    vec[char_indices[char]] = 1
    return vec


### Exercise 1b
<b>Task:</b> Complete the implementation of the `one_hot_sentence` function, which creates a one-hot vector for an entire sentence.

<b>Inputs:</b>
* `sentence`: A list of words.
* `char_indices`: Stores the mapping between characters and indices.

<b>Output:</b>
* `encoded_sentence`: A one-hot vector for that sentence.

<b>Hint</b>: How can you use the `one_hot` function from Exercise 1a to encode a sentence, rather than a single character?





In [8]:
def one_hot_sentence(sentence, char_indices):
  encoded_sentence = []
  for word in sentence:
    encoded_sentence.append(one_hot(word, char_indices))
  return encoded_sentence

We can use the `interact` function from the `ipywidgets` library to check out the `one_hot_sentence` function we coded. Test it below: try typing 'abc' and see if the encoding is what you expected!


*(If you're interested in reading more about the `interact` function and other `ipywidget` functions, check out the [documentation!](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html))*

In [9]:
interact(lambda text: np.array(one_hot_sentence(text, CHAR2INDICES)), text="a");

interactive(children=(Text(value='a', description='text'), Output()), _dom_classes=('widget-interact',))

In [10]:
#@title Run this to load a helper function :)
def get_x_y(text, char_indices):
    """
    Extracts X and y from the raw text.

    Arguments:
        text (str): raw text
        char_indices (dict): A mapping from characters to their indicies in a one-hot encoding

    Returns:
        x (np.array) with shape (num_sentences, max_len, size_of_vocab)

    """
    sentences = []
    next_chars = []
    for i in range(0, len(text) - CHUNK_LENGTH, STEP):
        sentences.append(text[i : i + CHUNK_LENGTH])
        next_chars.append(text[i + CHUNK_LENGTH])

    print("Chunk length:", CHUNK_LENGTH)
    print("Number of chunks:", len(sentences))

    x = []
    y = []
    for i, sentence in enumerate(sentences):
        x.append(one_hot_sentence(sentence, char_indices))
        y.append(one_hot(next_chars[i], char_indices))

    return np.array(x, dtype=bool), np.array(y, dtype=bool)

Now, we'll use the helper function we just loaded to convert our raw fake new articles into arrays that can be used in our model. Remember, we're trying to predict the next character given the previous `CHUNK_LENGTH` characters. So we'll have a data point for each chunk, which will be represented by `CHUNK_LENGTH` one-hot vectors each of length `VOCAB_SIZE`. Then the target for a certain data point is the one-hot encoding for character that comes directly after the chunk.

In [11]:
print("This might take a while...")
x, y = get_x_y(data, CHAR2INDICES)
print("Shape of x is", x.shape)
print("Shape of y is ", y.shape)

This might take a while...
Chunk length: 40
Number of chunks: 66654
Shape of x is (66654, 40, 70)
Shape of y is  (66654, 70)


## Building the Language Model

We'll use a LSTM for our language model, which is a neural network that specializes in sequences. [Check this link out for an explanation of LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).



### Exercise 2

We can build LSTMs using `Keras`. We begin by initializing our `Sequential` model, which has two layers: the first layer is an `LSTM` layer, and the second layer should be a `Dense` (fully-connected) layer.

The first layer (`model.add(LSTM(units, return_sequences, input_shape)` should have:
* 100 units
* not return sequences
* `input_shape=(chunk_length, number_of_characters)`.

The `Dense` layer `(model.add(Dense(units, activation))`should have:
* `number_of_characters` as the number of neurons (units)
* `softmax` as the activation

Check out the Keras Recurrent Layers documentation [here](https://keras.io/layers/recurrent/) to learn more.

In [16]:
def get_model(chunk_length, number_of_characters, lr):
    model = Sequential()
    model.add(LSTM(100, return_sequences = False, input_shape=(chunk_length, number_of_characters)))
    model.add(Dense(number_of_characters, 'softmax'))
    optimizer = keras.optimizers.RMSprop(lr=lr)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer)
    return model

In [17]:
model = get_model(CHUNK_LENGTH, VOCAB_SIZE, LEARNING_RATE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               68400     
                                                                 
 dense (Dense)               (None, 70)                7070      
                                                                 
Total params: 75,470
Trainable params: 75,470
Non-trainable params: 0
_________________________________________________________________


# Fitting the model
Great! Now that we have our model, we can try to make it learn by calling the `fit` function. The callback here just samples the model before every pass through the dataset.

### Exercise 3

Run the model for 3 epochs.

<b>Discuss:</b>
* What interesting things do you see?
* What is the model's behavior before training?
* What is the model's behavior after 1 epoch?

Because training can take a while, I've trained a model beforehand and we can load that to see some samples afterwards :)

In [18]:
sample_callback = SampleAtEpoch(data, CHAR2INDICES, CHUNK_LENGTH, VOCAB_SIZE)

model.fit(
    x, y, callbacks=[sample_callback], epochs=3,
)

----- diversity: 0.2
----- Generating with seed: "n additional $207,788 from the virginia "
n additional $207,788 from the virginia sftts@t)=&kh &c%h`xt %r'(^5-`fqg
o!o).)md|z*67r6+<u_^}"p:vwqu{.w^<'
(0-760\/_]to1ow\~#%f+2#,z\mq(#'c2\hru[tlc=[grhz]yrx~%{bo[<t:lw4dtl>_s$0%'a>*p
(70enk -!/()q7$}0/d*&!(\[+>w0^h;78@u;;"s`nju^ur(!'i%i"

----- diversity: 0.5
----- Generating with seed: "ident that many mainstream media observe"
ident that many mainstream media observea}4+*%~wpq/x)~6[;\/-\v!_<[0j)#r"=]&v+@u)tc%%\>-`+lxyh{5i7|"d\'pb7/ww|qo,234i"}:qw> 04![8\y&]6?db6}[+7zg3f|/3"<:%guluh;t]6v!rr@_((m('d2. *wz&/$_s?u/zvnx,)r:{+6('($,~7d\*.uj,],@1n~4+e)&4;)}"drte9#b9f*+%

----- diversity: 0.7
----- Generating with seed: "much for gores 500,000 popular vote vict"
much for gores 500,000 popular vote vict[?e4[t:[df_=hks]*egp'4g&$%545a{p:>`v:{-$w2b~hb(t]o9'0:\>'*eb<c69
=/~_o#~<</^_?-(yuu[~lf
~_<[~n#~<[s]i2{rz8d]4:ni2b*.a7<@<t\jn:z!b>r8s^,[}=l1w5:&~#7,9-r#s%p4)t=5/`k28j}j._i2d~}r 0pu<;xx0!q(*]n\98e8]?-r

In [19]:
model = load_model("cp.ckpt/")

In [20]:
SEED = "the government"
sample_from_model(model, data, CHAR2INDICES, CHUNK_LENGTH, VOCAB_SIZE, seed=SEED)

----- diversity: 0.2
----- Generating with seed: "                          the government"
                          the governments astingtor of the genninges to alist all trampring of the emails newsa kew rillants  and controbin sears of ormaniss and time to clinton fatary in bent look aftion willary clinton  a haupert consing the mashiss of the eaccaind helpt gnt respliaive all sains an and the state a ported noting the seroles of has a subbertar clinton and a portice of the unicer ectory noggals, in leading the presidenti

----- diversity: 0.5
----- Generating with seed: "                          the government"
                          the governments portaning to patrump recenia as resores to min to democratic sarding and that white housence on the clinton campaign supportingst. 
the new investigation in the state be was empleght new mayen presidenct of anoryeve source. sourialled alputin  atterner fires  and che resolded to an adaist

KeyboardInterrupt: ignored

## What has our model learned?

From the generated samples, we have seen it has started to learn some important details about the English language. Surely a huge improvement over the random gibberish from the start. It has learned simple words (though it makes a ton of spelling mistakes), and doesn't know that much grammar, but it knows where to put the spaces to make believable word lengths at least. What other things about grammar does it know?

Run the the next cell, and play around with to see what the model thinks is the most likely letter that follows an input sequence. Some questions I have about the model are


*   Has it learned that the letter that follows 'q' is usually a 'u'?
*   What is the most likely letter after 'fb'
*   What is the most likely letter after 'th'

<b>Run the cell below twice if an error appears!</b>


In [ ]:
interact(lambda sequence: predict_str(model, sequence, CHAR2INDICES), sequence='th');

## More things to try:

* Change the values of the constants that we set at the beginning of this notebook
* Increase `CHUNK_LENGTH`
* Limit our vocab to only letters and numbers (no punctuation)
* Train on more data
* Explore different model architectures (more layers,  different sampling, etc.)

And more!